In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd
import talib

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
idx = pd.IndexSlice

In [4]:
with pd.HDFStore('data.h5') as store:
    data = store['us/equities/monthly']
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 77788 entries, ('A', Timestamp('2006-12-31 00:00:00')) to ('ZION', Timestamp('2017-11-30 00:00:00'))
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   atr            77788 non-null  float64
 1   bb_down        77788 non-null  float64
 2   bb_high        77788 non-null  float64
 3   bb_low         77788 non-null  float64
 4   bb_mid         77788 non-null  float64
 5   bb_up          77788 non-null  float64
 6   macd           77788 non-null  float64
 7   natr           77788 non-null  float64
 8   rsi            77788 non-null  float64
 9   sector         77788 non-null  object 
 10  return_1m      77788 non-null  float64
 11  return_3m      77788 non-null  float64
 12  return_6m      77788 non-null  float64
 13  return_12m     77788 non-null  float64
 14  beta           77788 non-null  float64
 15  SMB            77788 non-null  float64
 16  HML            7778

In [5]:

before = len(data.index.unique('ticker').unique())

In [6]:
prices = (data.unstack('ticker')
        .sort_index()
        .ffill(limit=5)
        .dropna(axis=1)
        .stack('ticker')
        .swaplevel())
prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 45076 entries, ('AAPL', Timestamp('2002-01-31 00:00:00')) to ('ZION', Timestamp('2017-11-30 00:00:00'))
Data columns (total 27 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   atr            45076 non-null  float64
 1   bb_down        45076 non-null  float64
 2   bb_high        45076 non-null  float64
 3   bb_low         45076 non-null  float64
 4   bb_mid         45076 non-null  float64
 5   bb_up          45076 non-null  float64
 6   macd           45076 non-null  float64
 7   natr           45076 non-null  float64
 8   rsi            45076 non-null  float64
 9   sector         45076 non-null  object 
 10  return_1m      45076 non-null  float64
 11  return_3m      45076 non-null  float64
 12  return_6m      45076 non-null  float64
 13  return_12m     45076 non-null  float64
 14  beta           45076 non-null  float64
 15  SMB            45076 non-null  float64
 16  HML            4

In [7]:
after = len(prices.index.unique('ticker').unique())
print(f'Before: {before:,.0f} after: {after:,.0f}')

Before: 612 after: 236


In [8]:
intervals = [1, 5, 10, 21, 63]

In [11]:
returns = []
by_ticker = prices.groupby(level='ticker').close
for t in intervals:
    returns.append(by_ticker.pct_change(t).to_frame(f'ret_{t}'))
returns = pd.concat(returns, axis=1)

AttributeError: 'DataFrameGroupBy' object has no attribute 'close'

In [12]:
prices

,,atr,bb_down,bb_high,bb_low,bb_mid,bb_up,macd,natr,rsi,sector,...,RMW,CMA,momentum_3,momentum_6,momentum_3_6,momentum_12,momentum_3_12,year,month,target
ticker,date,,,,,,,,,,,,,,,,,,,,,
AAPL,2002-01-31,-0.789253,0.111199,0.951872,0.839854,0.895863,0.000819,-0.176197,3.863820,66.460045,Technology,...,-0.566646,-2.286294,-0.008019,-0.081991,-0.073972,-0.117538,-0.109519,2002.0,1.0,-0.122168
ABT,2002-01-31,-0.482813,0.055674,2.914318,2.848958,2.881638,0.009686,0.328044,1.755068,63.991991,Health Care,...,0.455584,0.914221,-0.008724,-0.025148,-0.016424,-0.016275,-0.007551,2002.0,1.0,-0.019931
ADBE,2002-01-31,-0.292206,0.016190,2.981229,2.861570,2.921399,0.103469,-0.086245,4.804679,47.069825,Technology,...,-0.967522,-0.679978,-0.000420,-0.102805,-0.102385,-0.106624,-0.106204,2002.0,1.0,0.079525
ADI,2002-01-31,0.646855,0.078831,3.582625,3.399639,3.491132,0.104155,-0.523424,4.892764,49.573563,Technology,...,-0.007432,0.377421,0.061779,0.005157,-0.056623,-0.016031,-0.077811,2002.0,1.0,-0.150457
ADM,2002-01-31,-0.913679,0.042903,2.422395,2.363418,2.392907,0.016074,-0.378651,2.683633,50.252706,Consumer Non-Durables,...,1.111548,0.242250,0.029956,0.036503,0.006547,0.023790,-0.006166,2002.0,1.0,-0.002684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XL,2017-11-30,0.228940,0.022826,3.725512,3.661543,3.693528,0.041143,-0.648420,2.288377,44.034569,Finance,...,0.506202,-0.601405,0.024875,0.023028,-0.001848,0.048589,0.023713,2017.0,11.0,-0.094281
XLNX,2017-11-30,0.577390,-0.003128,4.326784,4.258883,4.292833,0.071029,-0.350567,2.127503,39.733923,Technology,...,-0.003093,-0.897634,0.073838,0.064528,-0.009310,0.079433,0.005595,2017.0,11.0,-0.030068
XOM,2017-11-30,-0.216514,0.042662,4.450961,4.391602,4.421281,0.016698,-0.367920,0.929628,60.680304,Energy,...,0.288857,0.847070,0.030235,0.008042,-0.022192,-0.000095,-0.030329,2017.0,11.0,0.004202
